In [43]:
import sys, os
from pathlib import Path
sys.path.append("..")

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F
from PIL import Image
import numpy as np
import csv

from src.models.face_cnn import FaceCNN
from src.video_to_frames import extract_frames

device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
print("Using device:", device)


Using device: mps


In [44]:
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((48, 48)),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder("../data/raw/fer2013/train", transform=transform)
test_data  = datasets.ImageFolder("../data/raw/fer2013/test", transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_data,  batch_size=32, shuffle=False)

print("Classes:", train_data.classes)


Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [45]:
model = FaceCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    model.train()
    loss_sum = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        out = model(images)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
    print(f"Epoch {epoch+1} | Loss: {loss_sum/len(train_loader):.4f}")

# Save model ONCE
os.makedirs("../models", exist_ok=True)
torch.save(model.state_dict(), "../models/emotion_cnn.pth")
print("✅ Model saved!")


Epoch 1 | Loss: 1.6935
Epoch 2 | Loss: 1.4555
Epoch 3 | Loss: 1.3263
Epoch 4 | Loss: 1.2478
Epoch 5 | Loss: 1.1782
Epoch 6 | Loss: 1.1170
Epoch 7 | Loss: 1.0583
Epoch 8 | Loss: 0.9998
Epoch 9 | Loss: 0.9388
Epoch 10 | Loss: 0.8854
✅ Model saved!


In [46]:
model = FaceCNN().to(device)
model.load_state_dict(torch.load("../models/emotion_cnn.pth", map_location=device))
model.eval()
print("✅ Model loaded for inference")


✅ Model loaded for inference


In [47]:
def predict_emotion(frame_path):
    img = Image.open(frame_path)
    img = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        out = model(img)
        probs = F.softmax(out, dim=1).cpu().numpy()[0]
    return probs


In [48]:
extract_frames("../interview.mp4", "../data/frames/test1", fps=2)

frames = sorted(os.listdir("../data/frames/test1"))
timeline = np.array([predict_emotion(f"../data/frames/test1/{f}") for f in frames])

print("Timeline shape:", timeline.shape)

 Saved 15 frames to ../data/frames/test1
Timeline shape: (15, 7)


In [49]:
labels = ['angry','disgust','fear','happy','neutral','sad','surprise']
dominant = np.argmax(timeline, 1)
unique, counts = np.unique(dominant, return_counts=True)
ratio = dict(zip([labels[u] for u in unique], counts/len(dominant)))

transitions = np.sum(dominant[:-1] != dominant[1:])
volatility  = np.mean(np.abs(np.diff(timeline, axis=0)))
peak   = dict(zip(labels, timeline.max(0)))
var    = dict(zip(labels, timeline.var(0)))

features = []
for e in labels: features.append(ratio.get(e,0))
features += [transitions, volatility]
for e in labels: features.append(peak[e])
for e in labels: features.append(var[e])

X_sample = np.array(features, dtype=np.float32)
print("Feature shape:", X_sample.shape)

Feature shape: (23,)


In [50]:
label = 0  # change per video

row = list(X_sample) + [label]
file = "../data/deception_dataset.csv"
header = [f"f{i}" for i in range(23)] + ["label"]

if not os.path.exists(file):
    with open(file,"w") as f: csv.writer(f).writerow(header)

with open(file,"a") as f: csv.writer(f).writerow(row)
print("✅ saved to dataset")


✅ saved to dataset
